In [193]:
import pandas as pd
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
from pydeseq2.utils import load_example_data
import numpy as np
import pydeseq2

In [3]:
#metadata and count table
metadata = pd.read_table('../../../Comorbidity/adonis/metadata_12dataset_1280_adonis_number_gender.txt', sep = "\t")
metadata_CD = metadata.loc[(metadata['Study'] == 'Franzosa2019IBD_CD')]
metadata_CD = metadata_CD.set_index ('featureid')
metadata_CD = metadata_CD.sort_index()
table = pd.read_table('../../../Comorbidity/Meta_diseases_analyses/txt/CD_AD.txt', sep = "\t", skiprows=1)
table = table.set_index ('#OTU ID')
table = table[list(metadata_CD.index)]

In [371]:
table_20genes = table#.head(3800)
table_20genes = table_20genes.T
table_20genes = table_20genes.sort_index()
table_20genes.shape[1]

9087

In [439]:
table_20genes['ko:K00001'].values

array([3.78852100e+06, 4.28169600e+06, 5.37554900e+06, 5.80297700e+06,
       5.06215400e+06, 5.02339900e+06, 4.38105000e+06, 1.97721010e+07,
       2.71634410e+07, 2.18872810e+07, 6.28393900e+06, 6.25117600e+06,
       6.09745200e+06, 1.87568900e+06, 4.29505400e+06, 5.15308200e+06,
       1.03768130e+07, 3.53737300e+06, 4.56602900e+06, 4.01522000e+06,
       5.46079800e+06, 3.51220200e+06, 5.76315700e+06, 4.57539800e+06,
       7.33688000e+05, 2.72875800e+06, 9.41721900e+06, 7.22237200e+06,
       5.86067000e+06, 5.23160300e+06, 5.64566600e+06, 4.53519000e+05,
       3.15385200e+06, 3.32007700e+06, 4.83781100e+06, 3.35824900e+06,
       3.18962800e+06, 6.77545200e+06, 8.11358600e+06, 1.13167300e+07,
       1.21491200e+07, 3.34338400e+06, 4.04841800e+06, 4.73695600e+06,
       4.27558900e+06, 5.60396800e+06, 1.73499480e+07, 1.31994890e+07,
       1.33503070e+07, 9.80523800e+06, 9.17534200e+06, 1.16396620e+07,
       1.66045730e+07, 3.53920880e+07, 2.40282340e+07, 4.89963100e+06,
      

In [472]:
def pydeseq2_permutation_test(counts, group_labels, num_permutations=1000):
    """Permutation test for differential expression using PyDESeq2.

    Parameters
    ----------
    counts : array_like
        The count matrix of shape (n_samples, n_genes).
    group_labels : array_like
        The group labels of shape (n_samples,).
    num_permutations : int, optional (default=1000)
        The number of permutations to perform.

    Returns
    -------
    p_values : array_like
        The p-values of the permutation test for each gene.
    """
    # Convert counts and group labels to PyDESeq2 input format
    dds = DeseqDataSet(
    counts,
    group_labels,
    design_factors="disease",  # compare samples based on the "disease"
    # column ("B" vs "A")
    refit_cooks=True,
    n_cpus=8
)
    dds.deseq2()
    stat_res = DeseqStats(dds, n_cpus=8)
    # Get the test statistics for each gene
    res = stat_res.summary()
    res_df = stat_res.results_df
    sorted_df = res_df.sort_values(by='log2FoldChange', ascending=False)
    rankings = sorted_df['log2FoldChange'].rank(ascending=False)
    sorted_df['Rankings'] = rankings
    sorted_df.dropna()
    sorted_df = sorted_df.sort_index()
    ranks = sorted_df['Rankings']
    #initialize the null distribution of the ranking
    null_distribution = np.zeros((num_permutations, counts.shape[1]))

    # Perform the permutations
    for i in range(num_permutations):
        # Shuffle the group labels
        permuted_labels = np.random.permutation(group_labels['disease'])
        permuted_labels = pd.DataFrame(permuted_labels, index=group_labels.index)
        permuted_labels = pd.DataFrame(permuted_labels)
        permuted_labels.columns = ['disease']
        # Convert the permuted group labels to PyDESeq2 input format
        # Run the DESeq2 analysis with the permuted group labels
        permuted_labels.reindex(counts.index)
        #counts.reindex(permuted_labels.index)
        permuted_dds = DeseqDataSet(
        counts, 
        permuted_labels, 
        design_factors="disease")
        permuted_dds.deseq2()
        permuted_stat_res = DeseqStats(permuted_dds, n_cpus=8)
        # Get the test statistics for each gene with the permuted group labels
        permuted_res = permuted_stat_res.summary()
        res_df_permuted = permuted_stat_res.results_df
        sorted_df_permuted = res_df_permuted.sort_values(by='log2FoldChange', ascending=False)
        rankings_permuted = sorted_df_permuted['log2FoldChange'].rank(ascending=False)
        sorted_df_permuted['Rankings'] = rankings_permuted
        sorted_df_permuted.dropna()
        sorted_df_permuted = sorted_df_permuted.sort_index()
        #add ranks to the null distribution
        null_distribution[i,:] = sorted_df_permuted['Rankings'].values
        

    #TO DO: calculate the p values based on ranks
    p_values_rank = np.zeros(counts.shape[1])
    for i in range(counts.shape[1]):
        p_values_rank[i] = np.sum(null_distribution[:, i] > ranks[i]) / num_permutations

    return p_values_rank, sorted_df, null_distribution

In [470]:
np.zeros((3, 9000))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [473]:
p_values_rank, sorted_df, null_distribution = pydeseq2_permutation_test(table_20genes, metadata_CD, num_permutations=3)

Fitting size factors...
... done in 0.02 seconds.

Fitting dispersions...
... done in 1.09 seconds.

Fitting dispersion trend curve...
... done in 1.10 seconds.

Fitting MAP dispersions...
... done in 1.29 seconds.

Fitting LFCs...
... done in 0.55 seconds.

Refitting 3359 outlier genes.

Fitting dispersions...
... done in 0.39 seconds.

Fitting MAP dispersions...
... done in 0.37 seconds.

Fitting LFCs...
... done in 0.22 seconds.

Running Wald tests...
... done in 0.42 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.594560e+06,-0.266532,0.113514,-2.348017,1.887364e-02,3.970681e-02
ko:K00002,9.528195e+05,-0.299410,0.196004,-1.527567,1.266200e-01,2.059589e-01
ko:K00003,1.793023e+07,0.158491,0.126461,1.253273,2.101064e-01,3.137598e-01
ko:K00004,9.384642e+05,-0.745274,0.195983,-3.802757,1.430945e-04,4.439549e-04
ko:K00005,7.573665e+06,-0.405515,0.124744,-3.250772,1.150920e-03,3.125351e-03
...,...,...,...,...,...,...
ko:K22510,7.525708e+04,-2.113888,0.427544,-4.944259,7.643397e-07,3.181419e-06
ko:K22511,9.201190e+03,-3.341595,0.483578,-6.910153,4.841324e-12,5.473168e-11
ko:K22512,1.385335e+03,-2.701023,0.457407,-5.905071,3.524949e-09,2.621670e-08


Fitting size factors...
... done in 0.02 seconds.

Fitting dispersions...
... done in 1.09 seconds.

Fitting dispersion trend curve...
... done in 1.11 seconds.

Fitting MAP dispersions...
... done in 1.20 seconds.

Fitting LFCs...
... done in 0.53 seconds.

Refitting 4022 outlier genes.

Fitting dispersions...
... done in 0.47 seconds.

Fitting MAP dispersions...
... done in 0.46 seconds.

Fitting LFCs...
... done in 0.23 seconds.

Running Wald tests...
... done in 0.47 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.594560e+06,0.176987,0.114722,1.542739,0.122894,0.429336
ko:K00002,9.528195e+05,-0.168753,0.197102,-0.856168,0.391905,0.700534
ko:K00003,1.793023e+07,-0.058445,0.126862,-0.460696,0.645017,0.856531
ko:K00004,9.384642e+05,0.003736,0.206867,0.018060,0.985591,0.995972
ko:K00005,7.573665e+06,-0.080711,0.129991,-0.620893,0.534670,0.800346
...,...,...,...,...,...,...
ko:K22510,4.874396e+04,-0.802370,0.406695,-1.972906,0.048506,0.265831
ko:K22511,3.315484e+03,-0.930570,0.442950,-2.100846,0.035654,0.242949
ko:K22512,1.178028e+03,0.914258,0.486298,1.880035,0.060103,0.289802


Fitting size factors...
... done in 0.02 seconds.

Fitting dispersions...
... done in 1.10 seconds.

Fitting dispersion trend curve...
... done in 1.12 seconds.

Fitting MAP dispersions...
... done in 1.21 seconds.

Fitting LFCs...
... done in 0.52 seconds.

Refitting 3861 outlier genes.

Fitting dispersions...
... done in 0.40 seconds.

Fitting MAP dispersions...
... done in 0.40 seconds.

Fitting LFCs...
... done in 0.22 seconds.

Running Wald tests...
... done in 0.44 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.594560e+06,0.074807,0.115947,0.645180,5.188105e-01,7.899219e-01
ko:K00002,9.528195e+05,-0.034739,0.197788,-0.175638,8.605787e-01,9.596009e-01
ko:K00003,1.793023e+07,-0.107690,0.126754,-0.849594,3.955508e-01,6.840738e-01
ko:K00004,9.384642e+05,0.660479,0.198355,3.329786,8.691277e-04,4.664816e-03
ko:K00005,7.573665e+06,0.302260,0.127184,2.376553,1.747524e-02,6.818026e-02
...,...,...,...,...,...,...
ko:K22510,7.525708e+04,2.266504,0.424118,5.344040,9.089741e-08,2.323901e-06
ko:K22511,9.201190e+03,2.786674,0.499934,5.574086,2.488326e-08,8.333597e-07
ko:K22512,1.082537e+03,1.762101,0.462567,3.809400,1.393045e-04,8.707491e-04


Fitting size factors...
... done in 0.02 seconds.

Fitting dispersions...
... done in 1.12 seconds.

Fitting dispersion trend curve...
... done in 1.13 seconds.

Fitting MAP dispersions...
... done in 1.31 seconds.

Fitting LFCs...
... done in 0.75 seconds.

Refitting 4040 outlier genes.

Fitting dispersions...
... done in 0.44 seconds.

Fitting MAP dispersions...
... done in 0.43 seconds.

Fitting LFCs...
... done in 0.23 seconds.

Running Wald tests...
... done in 0.66 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.594560e+06,-0.151824,0.114942,-1.320877,0.186542,0.447850
ko:K00002,9.528195e+05,0.167876,0.197048,0.851955,0.394239,0.660080
ko:K00003,1.793023e+07,-0.128109,0.126286,-1.014436,0.310375,0.589011
ko:K00004,9.384642e+05,0.008847,0.206813,0.042778,0.965879,0.986609
ko:K00005,7.573665e+06,-0.124648,0.129569,-0.962019,0.336040,0.611625
...,...,...,...,...,...,...
ko:K22510,4.874396e+04,0.376238,0.410556,0.916412,0.359451,0.630220
ko:K22511,3.315484e+03,-0.038375,0.449210,-0.085427,0.931922,0.977354
ko:K22512,1.050363e+03,1.105719,0.474862,2.328505,0.019885,0.158855


In [474]:
p_values_rank

array([0.33333333, 0.66666667, 1.        , ..., 0.        , 0.        ,
       0.        ])

In [478]:
null_distribution

array([[2921. , 6294. , 4972. , ...,  935. ,  498. ,  505.5],
       [4759. , 5942. , 6787. , ..., 1107. ,  109. ,   68.5],
       [6038. , 3157. , 5828. , ...,  329. ,   81. ,   91.5]])

In [477]:
null_distribution[:,1]

array([6294., 5942., 3157.])

In [461]:
null_distribution[0:]

array([3914. , 2033. , 4718. , ..., 1949. ,  157. ,   29.5])

In [466]:
null_distribution.shape

(9087,)

In [385]:
    dds = DeseqDataSet(
    table_20genes,
    metadata_CD,
    design_factors="disease",  # compare samples based on the "disease"
    # column ("B" vs "A")
    refit_cooks=True,
    n_cpus=8
)
    dds.deseq2()
    stat_res = DeseqStats(dds, n_cpus=8)
    # Get the test statistics for each gene
    res = stat_res.summary()
    #extract the log2FoldChange for each feature
    log2_fold_changes = stat_res.results_df["log2FoldChange"]
    #sort the log2foldchanges in descending order
    sorted_indices = np.argsort(log2_fold_changes)[::-1]
    sorted_log2_fold_changes = log2_fold_changes[sorted_indices]
    #calculate the rank of each gene
    ranks = np.argsort(sorted_log2_fold_changes)

Fitting size factors...
... done in 0.02 seconds.

Fitting dispersions...
... done in 1.10 seconds.

Fitting dispersion trend curve...
... done in 1.19 seconds.

Fitting MAP dispersions...
... done in 1.30 seconds.

Fitting LFCs...
... done in 0.52 seconds.

Refitting 3359 outlier genes.

Fitting dispersions...
... done in 0.41 seconds.

Fitting MAP dispersions...
... done in 0.39 seconds.

Fitting LFCs...
... done in 0.22 seconds.

Running Wald tests...
... done in 0.37 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.594560e+06,-0.266532,0.113514,-2.348017,1.887364e-02,3.970681e-02
ko:K00002,9.528195e+05,-0.299410,0.196004,-1.527567,1.266200e-01,2.059589e-01
ko:K00003,1.793023e+07,0.158491,0.126461,1.253273,2.101064e-01,3.137598e-01
ko:K00004,9.384642e+05,-0.745274,0.195983,-3.802757,1.430945e-04,4.439549e-04
ko:K00005,7.573665e+06,-0.405515,0.124744,-3.250772,1.150920e-03,3.125351e-03
...,...,...,...,...,...,...
ko:K22510,7.525708e+04,-2.113888,0.427544,-4.944259,7.643397e-07,3.181419e-06
ko:K22511,9.201190e+03,-3.341595,0.483578,-6.910153,4.841324e-12,5.473168e-11
ko:K22512,1.385335e+03,-2.701023,0.457407,-5.905071,3.524949e-09,2.621670e-08


In [402]:
    log2_fold_changes = stat_res.results_df["log2FoldChange"]
    #sort the log2foldchanges in descending order
    sorted_indices = np.argsort(log2_fold_changes)[::-1]
    sorted_log2_fold_changes = log2_fold_changes[sorted_indices]
    #calculate the rank of each gene
    ranks = np.argsort(sorted_log2_fold_changes)

In [428]:
import pandas as pd

# create a sample dataframe
df = stat_res.results_df
# sort the dataframe by the 'Salary' column in descending order
sorted_df = df.sort_values(by='log2FoldChange', ascending=False)

# get the rankings of the index
rankings = sorted_df['log2FoldChange'].rank(ascending=False)

# add the rankings as a new column in the dataframe
sorted_df['Rankings'] = rankings

# print the sorted dataframe with rankings
sorted_df.dropna()


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Rankings
#OTU ID,,,,,,,
ko:K06072,16830.296583,10.864397,0.702399,15.467564,5.744029e-54,5.214430e-50,1.0
ko:K20228,240.355768,9.461858,1.228198,7.703855,1.320214e-14,2.010889e-13,2.0
ko:K13809,304.854499,8.889919,1.634456,5.439069,5.355983e-08,2.753206e-07,3.5
ko:K00816,304.854499,8.889919,1.634456,5.439069,5.355983e-08,2.753206e-07,3.5
ko:K08825,8.730474,8.069311,1.868972,4.317514,1.577965e-05,5.545785e-05,6.0
...,...,...,...,...,...,...,...
ko:K20762,2114.815747,-5.090430,0.551211,-9.234997,2.582944e-20,6.354462e-19,9076.0
ko:K07756,1061.452622,-5.165220,0.700784,-7.370626,1.698285e-13,2.364575e-12,9077.0
ko:K12519,12725.155654,-5.890995,0.513378,-11.474966,1.762462e-30,8.420856e-28,9078.0


In [430]:
b.sort_index()

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Rankings
#OTU ID,,,,,,,
ko:K00001,6.594560e+06,-0.266532,0.113514,-2.348017,1.887364e-02,3.970681e-02,4939.0
ko:K00002,9.528195e+05,-0.299410,0.196004,-1.527567,1.266200e-01,2.059589e-01,5040.0
ko:K00003,1.793023e+07,0.158491,0.126461,1.253273,2.101064e-01,3.137598e-01,2621.0
ko:K00004,9.384642e+05,-0.745274,0.195983,-3.802757,1.430945e-04,4.439549e-04,6217.0
ko:K00005,7.573665e+06,-0.405515,0.124744,-3.250772,1.150920e-03,3.125351e-03,5401.0
...,...,...,...,...,...,...,...
ko:K22510,7.525708e+04,-2.113888,0.427544,-4.944259,7.643397e-07,3.181419e-06,7980.0
ko:K22511,9.201190e+03,-3.341595,0.483578,-6.910153,4.841324e-12,5.473168e-11,8948.0
ko:K22512,1.385335e+03,-2.701023,0.457407,-5.905071,3.524949e-09,2.621670e-08,8735.0


In [426]:
a.shape

(9078, 7)

In [427]:
b.shape

(9087, 7)

In [411]:
ranks

#OTU ID
ko:K06049    6784
ko:K20201    2270
ko:K13795    4564
ko:K00814     878
ko:K08794    3997
             ... 
ko:K20707    3497
ko:K07751    4549
ko:K12513    3224
ko:K12080    8293
ko:K12083    8511
Name: log2FoldChange, Length: 9087, dtype: int64

In [401]:
ranks_sorted = ranks.sort_index()
ranks_sorted.index

Index(['ko:K00001', 'ko:K00002', 'ko:K00003', 'ko:K00004', 'ko:K00005',
       'ko:K00006', 'ko:K00007', 'ko:K00008', 'ko:K00009', 'ko:K00010',
       ...
       'ko:K22505', 'ko:K22506', 'ko:K22507', 'ko:K22514', 'ko:K22514',
       'ko:K22514', 'ko:K22514', 'ko:K22514', 'ko:K22514', 'ko:K22514'],
      dtype='object', name='#OTU ID', length=9087)

In [384]:
p_values_rank, df = pydeseq2_permutation_test(table_20genes, metadata_CD, num_permutations=1)

Fitting size factors...
... done in 0.02 seconds.

Fitting dispersions...
... done in 1.13 seconds.

Fitting dispersion trend curve...
... done in 1.18 seconds.

Fitting MAP dispersions...
... done in 1.26 seconds.

Fitting LFCs...
... done in 0.52 seconds.

Refitting 3359 outlier genes.

Fitting dispersions...
... done in 0.38 seconds.

Fitting MAP dispersions...
... done in 0.38 seconds.

Fitting LFCs...
... done in 0.42 seconds.

Running Wald tests...
... done in 0.36 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.594560e+06,-0.266532,0.113514,-2.348017,1.887364e-02,3.970681e-02
ko:K00002,9.528195e+05,-0.299410,0.196004,-1.527567,1.266200e-01,2.059589e-01
ko:K00003,1.793023e+07,0.158491,0.126461,1.253273,2.101064e-01,3.137598e-01
ko:K00004,9.384642e+05,-0.745274,0.195983,-3.802757,1.430945e-04,4.439549e-04
ko:K00005,7.573665e+06,-0.405515,0.124744,-3.250772,1.150920e-03,3.125351e-03
...,...,...,...,...,...,...
ko:K22510,7.525708e+04,-2.113888,0.427544,-4.944259,7.643397e-07,3.181419e-06
ko:K22511,9.201190e+03,-3.341595,0.483578,-6.910153,4.841324e-12,5.473168e-11
ko:K22512,1.385335e+03,-2.701023,0.457407,-5.905071,3.524949e-09,2.621670e-08


/var/folders/0g/jffzlx7x0v37frp3q4ssdnp80000gn/T/ipykernel_64714/1410331532.py:38: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  ranks = ranks.reindex(counts.index)


ValueError: cannot reindex on an axis with duplicate labels

In [ ]:
p_values_rank

In [256]:
a = table_20genes.T.drop_duplicates().T

In [257]:
set(table_20genes.columns) - a.colums

AttributeError: 'DataFrame' object has no attribute 'colums'

In [135]:
permuted_labels = np.random.permutation(metadata_CD['disease'])
permuted_labels = pd.DataFrame(permuted_labels, index=metadata_CD.index)
permuted_labels = pd.DataFrame(permuted_labels)
permuted_labels.columns = ['disease']
permuted_labels

,disease
featureid,
SRR6468502,CD
SRR6468505,CD
SRR6468507,CD
SRR6468514,Healthy
SRR6468517,CD
...,...
SRR6468707,Healthy
SRR6468709,CD
SRR6468711,Healthy


In [195]:
import pandas as pd
import numpy as np
import pydeseq2

# Load your data as a pandas dataframe
counts = table_20genes
metadata = metadata_CD

# Create a DESeqDataSet object from the counts and metadata
dds = DeseqDataSet(counts, metadata, design_factors="disease")

# Run the differential expression analysis
dds = pydeseq2.DESeq(dds, name='permute', design='~condition', n=100)
res = pydeseq2.results(dds)

# Get the log2 fold changes and rank them
log2fc = res['log2FoldChange']
ranked_log2fc = np.argsort(np.abs(log2fc))[::-1]

# Calculate p-values based on the ranking of log2 fold changes
p_values = [(np.where(ranked_log2fc == i)[0][0] + 1) / len(ranked_log2fc) for i in range(len(log2fc))]

# Add the p-values to the results dataframe
res['p_values'] = p_values


AttributeError: module 'pydeseq2' has no attribute 'test'

In [353]:
import pydeseq2

def pydeseq2_permutation_test(counts, group_labels, num_permutations=10):
    """Permutation test for differential expression using PyDESeq2.

    Parameters
    ----------
    counts : array_like
        The count matrix of shape (n_samples, n_genes).
    group_labels : array_like
        The group labels of shape (n_samples,).
    num_permutations : int, optional (default=1000)
        The number of permutations to perform.

    Returns
    -------
    p_values : array_like
        The p-values of the permutation test for each gene.
    """
    # Convert counts and group labels to PyDESeq2 input format
    dds = DeseqDataSet(
    counts,
    group_labels,
    design_factors="disease",  # compare samples based on the "disease"
    # column ("B" vs "A")
    refit_cooks=True#,
    #n_cpus=8,
)
    dds.deseq2()
    stat_res = DeseqStats(dds, n_cpus=8)
    # Get the test statistics for each gene
    res = stat_res.summary()
    #extract the log2FoldChange for each feature
    log2_fold_changes = stat_res.results_df["log2FoldChange"]
    #sort the log2foldchanges in descending order
    sorted_indices = np.argsort(log2_fold_changes)[::-1]
    sorted_log2_fold_changes = log2_fold_changes[sorted_indices]
    #calculate the rank of each gene
    ranks = np.argsort(sorted_log2_fold_changes)
    
    return ranks
    #initialize the null distribution of the ranking
    null_distribution = np.zeros((num_permutations, counts.shape[1]))

    # Perform the permutations
    for i in range(num_permutations):
        # Shuffle the group labels
        permuted_labels = np.random.permutation(group_labels['disease'])
        permuted_labels = pd.DataFrame(permuted_labels, index=group_labels.index)
        permuted_labels = pd.DataFrame(permuted_labels)
        permuted_labels.columns = ['disease']
        # Convert the permuted group labels to PyDESeq2 input format
        # Run the DESeq2 analysis with the permuted group labels
        permuted_labels.reindex(counts.index)
        #counts.reindex(permuted_labels.index)
        permuted_dds = DeseqDataSet(
        counts, 
        permuted_labels, 
        design_factors="disease")
        permuted_dds.deseq2()
        permuted_stat_res = DeseqStats(permuted_dds, n_cpus=8)
        # Get the test statistics for each gene with the permuted group labels
        permuted_res = permuted_stat_res.summary()
        log2_fold_changes_permutated = permuted_stat_res.results_df["log2FoldChange"]
        sorted_indices_permutated = np.argsort(log2_fold_changes_permutated)[::-1]
        sorted_log2_fold_changes_permutated = log2_fold_changes_permutated[sorted_indices_permutated]
        ranks_permutated_ori = np.argsort(sorted_log2_fold_changes_permutated)
        ranks_permutated = ranks_permutated_ori.reindex(ranks.index)
        #add ranks to the null distribution
        null_distribution[i,:] = ranks_permutated

    #TO DO: calculate the p values based on ranks
    p_values_rank = np.zeros(counts.shape[1])
    for i in range(counts.shape[1]):
        p_values_rank[i] = np.sum(null_distribution[:, i] > ranks[i]) / num_permutations

    #return p_values_rank, stat_res.results_df,ranks_permutated_ori

In [364]:

# extract the log2FoldChange for each feature
#     log2_fold_changes = stat_res.results_df["log2FoldChange"]
#     #sort the log2foldchanges in descending order
#     sorted_indices = np.argsort(log2_fold_changes)[::-1]
#     sorted_log2_fold_changes = log2_fold_changes[sorted_indices]
#     #calculate the rank of each gene
#     ranks = np.argsort(sorted_log2_fold_changes)


log2_fold_changes = stat_res.results_df['log2FoldChange']
sorted_indices = np.argsort(log2_fold_changes)[::-1]
sorted_indices
sorted_log2_fold_changes = log2_fold_changes[sorted_indices]
sorted_log2_fold_changes
ranks = np.argsort(sorted_log2_fold_changes)
ranks

#OTU ID
ko:K06049     509
ko:K04656    2846
ko:K02488     666
ko:K03621     392
ko:K00648    3115
             ... 
ko:K03317    1449
ko:K01720     840
ko:K02594    2494
ko:K05358     233
ko:K00433    3341
Name: log2FoldChange, Length: 3500, dtype: int64

In [365]:
ranks = pydeseq2_permutation_test(table_20genes, metadata_CD, num_permutations=1)
ranks

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.47 seconds.

Fitting dispersion trend curve...
... done in 0.45 seconds.

Fitting MAP dispersions...
... done in 0.53 seconds.

Fitting LFCs...
... done in 0.19 seconds.

Refitting 967 outlier genes.

Fitting dispersions...
... done in 0.12 seconds.

Fitting MAP dispersions...
... done in 0.12 seconds.

Fitting LFCs...
... done in 0.06 seconds.

Running Wald tests...
... done in 0.33 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.981823e+06,-0.473368,0.134195,-3.527472,4.195483e-04,1.227615e-03
ko:K00002,9.337573e+05,-0.286306,0.191473,-1.495281,1.348411e-01,2.152697e-01
ko:K00003,1.737498e+07,0.163152,0.112288,1.452977,1.462301e-01,2.299717e-01
ko:K00004,9.189208e+05,-0.755815,0.190660,-3.964199,7.364268e-05,2.436367e-04
ko:K00005,7.323692e+06,-0.387899,0.107804,-3.598183,3.204477e-04,9.590706e-04
...,...,...,...,...,...,...
ko:K07129,9.987518e+04,2.698008,0.304045,8.873720,7.074216e-19,1.500998e-17
ko:K07130,1.427994e+06,0.295340,0.189754,1.556436,1.196044e-01,1.951278e-01
ko:K07131,1.140200e+03,-1.212958,0.249800,-4.855719,1.199507e-06,4.909188e-06


#OTU ID
ko:K06049     575
ko:K04656    3102
ko:K00814     748
ko:K02488     446
ko:K03621    3395
             ... 
ko:K03317    1005
ko:K01720    2722
ko:K02594    1339
ko:K05358     265
ko:K00433    3642
Name: log2FoldChange, Length: 3800, dtype: int64

In [366]:
ranks.drop_duplicates()

#OTU ID
ko:K06049     575
ko:K04656    3102
ko:K00814     748
ko:K02488     446
ko:K03621    3395
             ... 
ko:K03317    1005
ko:K01720    2722
ko:K02594    1339
ko:K05358     265
ko:K00433    3642
Name: log2FoldChange, Length: 3799, dtype: int64

In [369]:
rank.loc[ranks.duplicated()]

/var/folders/0g/jffzlx7x0v37frp3q4ssdnp80000gn/T/ipykernel_64714/1805765138.py:1: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  rank.loc[ranks.duplicated()]


ValueError: cannot reindex on an axis with duplicate labels

In [361]:
len(list(ranks.index))

3800

In [333]:
df = table_20genes.T
mask = df.index.duplicated()
unique_index = df.index[~mask]
unique_df = df[~mask]

In [336]:
len(unique_df)
#table_20genes
unique_df

,SRR6468502,SRR6468505,SRR6468507,SRR6468514,SRR6468517,SRR6468518,SRR6468520,SRR6468522,SRR6468523,SRR6468525,...,SRR6468686,SRR6468687,SRR6468689,SRR6468695,SRR6468706,SRR6468707,SRR6468709,SRR6468711,SRR6468712,SRR6468713
#OTU ID,,,,,,,,,,,,,,,,,,,,,
ko:K00001,3788521.0,4281696.0,5375549.0,5802977.0,5062154.0,5023399.0,4381050.0,19772101.0,27163441.0,21887281.0,...,5206907.0,11782853.0,25316593.0,21467617.0,3815064.0,2659998.0,3577829.0,2185426.0,5059494.0,3672913.0
ko:K00002,4252331.0,558050.0,323271.0,617821.0,779621.0,474776.0,429502.0,1277663.0,3159620.0,1498928.0,...,334567.0,1078344.0,2857260.0,4095245.0,860591.0,864282.0,409632.0,168015.0,793229.0,1774330.0
ko:K00003,21510389.0,8775543.0,20194996.0,18315103.0,19144228.0,11208863.0,16107150.0,45832461.0,49445669.0,30563184.0,...,11468152.0,28898405.0,53244149.0,49694762.0,11376941.0,16735436.0,13185827.0,4972053.0,7240698.0,11859648.0
ko:K00004,2037312.0,186098.0,543147.0,373672.0,678775.0,414967.0,335638.0,1270881.0,1719930.0,1822690.0,...,484952.0,810462.0,1804752.0,8212734.0,391578.0,109128.0,150511.0,198549.0,946442.0,1634684.0
ko:K00005,18760322.0,2619603.0,5555903.0,8324607.0,10436428.0,4347966.0,3581777.0,15852516.0,14909116.0,9825915.0,...,4082267.0,8534685.0,15659660.0,12274251.0,3358789.0,7663078.0,4180944.0,2859093.0,3097564.0,7378254.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ko:K22510,223567.0,861.0,39755.0,4153.0,3030.0,2465.0,1886.0,19984.0,8918.0,17536.0,...,831.0,42294.0,6183.0,3043.0,603.0,6487.0,6621.0,74657.0,5607.0,276701.0
ko:K22511,29347.0,221.0,1690.0,23.0,108.0,73.0,157.0,837.0,786.0,1672.0,...,393.0,3658.0,1617.0,328.0,61.0,163.0,317.0,3969.0,249.0,21430.0
ko:K22512,18290.0,181.0,125.0,11.0,62.0,44.0,71.0,284.0,301.0,748.0,...,364.0,560.0,1372.0,169.0,37.0,28.0,17.0,665.0,142.0,6311.0


In [338]:
table_20genes.T

,SRR6468502,SRR6468505,SRR6468507,SRR6468514,SRR6468517,SRR6468518,SRR6468520,SRR6468522,SRR6468523,SRR6468525,...,SRR6468686,SRR6468687,SRR6468689,SRR6468695,SRR6468706,SRR6468707,SRR6468709,SRR6468711,SRR6468712,SRR6468713
#OTU ID,,,,,,,,,,,,,,,,,,,,,
ko:K00001,3788521.0,4281696.0,5375549.0,5802977.0,5062154.0,5023399.0,4381050.0,19772101.0,27163441.0,21887281.0,...,5206907.0,11782853.0,25316593.0,21467617.0,3815064.0,2659998.0,3577829.0,2185426.0,5059494.0,3672913.0
ko:K00002,4252331.0,558050.0,323271.0,617821.0,779621.0,474776.0,429502.0,1277663.0,3159620.0,1498928.0,...,334567.0,1078344.0,2857260.0,4095245.0,860591.0,864282.0,409632.0,168015.0,793229.0,1774330.0
ko:K00003,21510389.0,8775543.0,20194996.0,18315103.0,19144228.0,11208863.0,16107150.0,45832461.0,49445669.0,30563184.0,...,11468152.0,28898405.0,53244149.0,49694762.0,11376941.0,16735436.0,13185827.0,4972053.0,7240698.0,11859648.0
ko:K00004,2037312.0,186098.0,543147.0,373672.0,678775.0,414967.0,335638.0,1270881.0,1719930.0,1822690.0,...,484952.0,810462.0,1804752.0,8212734.0,391578.0,109128.0,150511.0,198549.0,946442.0,1634684.0
ko:K00005,18760322.0,2619603.0,5555903.0,8324607.0,10436428.0,4347966.0,3581777.0,15852516.0,14909116.0,9825915.0,...,4082267.0,8534685.0,15659660.0,12274251.0,3358789.0,7663078.0,4180944.0,2859093.0,3097564.0,7378254.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ko:K22510,223567.0,861.0,39755.0,4153.0,3030.0,2465.0,1886.0,19984.0,8918.0,17536.0,...,831.0,42294.0,6183.0,3043.0,603.0,6487.0,6621.0,74657.0,5607.0,276701.0
ko:K22511,29347.0,221.0,1690.0,23.0,108.0,73.0,157.0,837.0,786.0,1672.0,...,393.0,3658.0,1617.0,328.0,61.0,163.0,317.0,3969.0,249.0,21430.0
ko:K22512,18290.0,181.0,125.0,11.0,62.0,44.0,71.0,284.0,301.0,748.0,...,364.0,560.0,1372.0,169.0,37.0,28.0,17.0,665.0,142.0,6311.0


In [221]:
#extract the log2FoldChange for each feature
log2_fold_changes = res_df["log2FoldChange"]
#sort the log2foldchanges in descending order
sorted_indices = np.argsort(log2_fold_changes)[::-1]
sorted_log2_fold_changes = log2_fold_changes[sorted_indices]
sorted_log2_fold_changes
# #calculate the rank of each gene
ranks = np.argsort(sorted_log2_fold_changes)
ranks

#OTU ID
ko:K00011    39
ko:K00039    38
ko:K00045    37
ko:K00021    36
ko:K00043    35
ko:K00034    34
ko:K00006    33
ko:K00003    32
ko:K00030    31
ko:K00013    30
ko:K00018    29
ko:K00031    28
ko:K00009    27
ko:K00012    26
ko:K00014    25
ko:K00027    24
ko:K00008    23
ko:K00029    22
ko:K00048    21
ko:K00041    20
ko:K00010    19
ko:K00024    18
ko:K00016    17
ko:K00046    16
ko:K00002    15
ko:K00040    14
ko:K00005    13
ko:K00023    12
ko:K00001    11
ko:K00042    10
ko:K00035     9
ko:K00015     8
ko:K00019     7
ko:K00004     6
ko:K00020     5
ko:K00038     4
ko:K00033     3
ko:K00036     2
ko:K00007     1
ko:K00032     0
Name: log2FoldChange, dtype: int64

In [222]:
np.argsort(sorted_indices)

#OTU ID
ko:K00048    28
ko:K00046    24
ko:K00045     7
ko:K00043    33
ko:K00042    26
ko:K00041     6
ko:K00040    38
ko:K00039    16
ko:K00038    12
ko:K00036    20
ko:K00035     0
ko:K00034    13
ko:K00033     9
ko:K00032    14
ko:K00031    31
ko:K00030    22
ko:K00029    10
ko:K00027    32
ko:K00024    34
ko:K00023     3
ko:K00021    27
ko:K00020    21
ko:K00019    15
ko:K00018    17
ko:K00016     8
ko:K00015    11
ko:K00014    39
ko:K00013    36
ko:K00012     5
ko:K00011    30
ko:K00010    37
ko:K00009    35
ko:K00008     1
ko:K00007    25
ko:K00006    19
ko:K00005    29
ko:K00004     4
ko:K00003     2
ko:K00002    23
ko:K00001    18
Name: log2FoldChange, dtype: int64

In [197]:
ranks_permutated = np.argsort(sorted_log2_fold_changes_permutated)
ranks_permutated

#OTU ID
ko:K00011    39
ko:K00039    38
ko:K00045    37
ko:K00021    36
ko:K00043    35
ko:K00034    34
ko:K00006    33
ko:K00003    32
ko:K00030    31
ko:K00013    30
ko:K00018    29
ko:K00031    28
ko:K00009    27
ko:K00012    26
ko:K00014    25
ko:K00027    24
ko:K00008    23
ko:K00029    22
ko:K00048    21
ko:K00041    20
ko:K00010    19
ko:K00024    18
ko:K00016    17
ko:K00046    16
ko:K00002    15
ko:K00040    14
ko:K00005    13
ko:K00023    12
ko:K00001    11
ko:K00042    10
ko:K00035     9
ko:K00015     8
ko:K00019     7
ko:K00004     6
ko:K00020     5
ko:K00038     4
ko:K00033     3
ko:K00036     2
ko:K00007     1
ko:K00032     0
Name: log2FoldChange, dtype: int64

In [179]:
p_values_rank, res_df , null_dis, ranks, ranks_permutated = pydeseq2_permutation_test(table_20genes, metadata_CD, num_permutations=10)

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.02 seconds.

Fitting dispersion trend curve...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.02 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Refitting 6 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.01 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.746768e+06,-0.349725,0.122449,-2.856089,4.288944e-03,1.225413e-02
ko:K00002,9.121744e+05,-0.200425,0.187151,-1.070928,2.842020e-01,3.552525e-01
ko:K00003,1.745186e+07,0.212549,0.119082,1.784892,7.427876e-02,1.414834e-01
ko:K00004,8.590762e+05,-0.621447,0.173664,-3.578439,3.456520e-04,1.382608e-03
ko:K00005,7.148575e+06,-0.269486,0.102661,-2.624998,8.664938e-03,2.166235e-02
ko:K00006,2.778698e+01,0.524251,1.193842,0.439129,6.605681e-01,7.339646e-01
ko:K00007,1.624141e+04,-1.683054,0.382272,-4.402769,1.068782e-05,6.107325e-05
ko:K00008,1.203914e+07,0.032126,0.103729,0.309714,7.567785e-01,8.181389e-01
ko:K00009,6.161752e+06,0.151470,0.128670,1.177200,2.391157e-01,3.188210e-01


Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.40 seconds.

Fitting dispersion trend curve...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.02 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Refitting 8 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.12 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.746768e+06,0.140198,0.125999,1.112692,0.265841,0.625508
ko:K00002,9.121744e+05,0.109014,0.188104,0.579542,0.562223,0.886947
ko:K00003,1.745186e+07,0.012894,0.120742,0.106786,0.914959,0.938419
ko:K00004,8.590762e+05,0.042332,0.182511,0.231941,0.816583,0.938419
ko:K00005,7.148575e+06,0.132394,0.103814,1.275300,0.202203,0.625508
ko:K00006,2.778698e+01,0.259467,1.203926,0.215517,0.829364,0.938419
ko:K00007,1.624141e+04,0.576905,0.403672,1.429144,0.152963,0.625508
ko:K00008,1.203914e+07,0.139041,0.103084,1.348811,0.177398,0.625508
ko:K00009,6.161752e+06,-0.068107,0.129410,-0.526286,0.598689,0.886947


Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.38 seconds.

Fitting dispersion trend curve...
... done in 0.03 seconds.

Fitting MAP dispersions...
... done in 0.02 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Refitting 8 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.14 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


...

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.746768e+06,-0.019404,0.126655,-0.153203,0.878239,0.946091
ko:K00002,9.121744e+05,0.135669,0.187904,0.722013,0.470286,0.783811
ko:K00003,1.745186e+07,-0.025053,0.120773,-0.207436,0.835669,0.946091
ko:K00004,8.590762e+05,0.354479,0.179754,1.972022,0.048607,0.388856
ko:K00005,7.148575e+06,-0.130829,0.105110,-1.244689,0.213246,0.616052
ko:K00006,2.778698e+01,1.144258,1.191110,0.960666,0.336720,0.664518
ko:K00007,1.165235e+04,0.151062,0.373543,0.404404,0.685916,0.885052
ko:K00008,1.203914e+07,-0.068342,0.103728,-0.658859,0.509986,0.784595
ko:K00009,6.161752e+06,-0.037686,0.129538,-0.290923,0.771110,0.946091


Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.35 seconds.

Fitting dispersion trend curve...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.02 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Refitting 8 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.15 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.746768e+06,-0.291094,0.123872,-2.349949,0.018776,0.399505
ko:K00002,9.121744e+05,0.064834,0.188269,0.344368,0.730569,0.970183
ko:K00003,1.745186e+07,-0.065455,0.120607,-0.542717,0.587325,0.970183
ko:K00004,8.590762e+05,-0.127719,0.182204,-0.700969,0.483323,0.970183
ko:K00005,7.148575e+06,-0.023374,0.105765,-0.221000,0.825092,0.970183
ko:K00006,2.778698e+01,-0.995451,1.196057,-0.832278,0.405252,0.953535
ko:K00007,1.880564e+04,-0.798238,0.414925,-1.923814,0.054378,0.543779
ko:K00008,1.203914e+07,0.048141,0.103796,0.463803,0.642789,0.970183
ko:K00009,6.161752e+06,-0.127849,0.129044,-0.990741,0.321812,0.831480


Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.02 seconds.

Fitting dispersion trend curve...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.02 seconds.

Fitting LFCs...
... done in 0.34 seconds.

Refitting 8 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.36 seconds.

Running Wald tests...
... done in 0.13 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.746768e+06,0.299771,0.123672,2.423912,1.535433e-02,9.387079e-02
ko:K00002,9.121744e+05,0.090799,0.188159,0.482567,6.294030e-01,9.239028e-01
ko:K00003,1.745186e+07,0.113013,0.120301,0.939420,3.475149e-01,8.687873e-01
ko:K00004,8.590762e+05,0.113998,0.180646,0.631056,5.280037e-01,9.239028e-01
ko:K00005,7.148575e+06,0.126998,0.105113,1.208202,2.269697e-01,6.983682e-01
ko:K00006,2.778698e+01,-1.218652,1.191906,-1.022439,3.065732e-01,8.175284e-01
ko:K00007,1.880564e+04,-1.346011,0.405770,-3.317179,9.093124e-04,1.818625e-02
ko:K00008,1.203914e+07,-0.000596,0.103873,-0.005741,9.954197e-01,9.954197e-01
ko:K00009,6.161752e+06,0.037262,0.129500,0.287739,7.735465e-01,9.239028e-01


In [177]:
log2_fold_changes_permutated = res_df["log2FoldChange"]
sorted_indices_permutated = np.argsort(log2_fold_changes_permutated)[::-1]
sorted_log2_fold_changes_permutated = log2_fold_changes_permutated[sorted_indices_permutated]
sorted_log2_fold_changes_permutated
#ranks_permutated = sorted_log2_fold_changes_permutated.reindex(ranks.index)


#OTU ID
ko:K00011    2.940580
ko:K00039    1.639975
ko:K00045    1.525989
ko:K00021    1.133299
ko:K00043    0.817794
ko:K00034    0.537521
ko:K00006    0.524251
ko:K00003    0.212549
ko:K00030    0.179650
ko:K00013    0.176822
ko:K00018    0.159959
ko:K00031    0.153504
ko:K00009    0.151470
ko:K00012    0.108391
ko:K00014    0.081653
ko:K00027    0.063740
ko:K00008    0.032126
ko:K00029    0.003182
ko:K00048   -0.001925
ko:K00041   -0.016957
ko:K00010   -0.061962
ko:K00024   -0.114817
ko:K00016   -0.170764
ko:K00046   -0.182340
ko:K00002   -0.200425
ko:K00040   -0.216467
ko:K00005   -0.269486
ko:K00023   -0.293838
ko:K00001   -0.349725
ko:K00042   -0.413456
ko:K00035   -0.487858
ko:K00015   -0.528415
ko:K00019   -0.604552
ko:K00004   -0.621447
ko:K00020   -0.629800
ko:K00038   -0.739097
ko:K00033   -0.881720
ko:K00036   -0.898189
ko:K00007   -1.683054
ko:K00032   -2.690234
Name: log2FoldChange, dtype: float64

In [174]:
null_dis

array([[ 4.98428760e-01, -1.65600728e-03,  6.37736996e-03,
        -8.06657403e-02, -1.64281285e+00, -8.13884581e-02,
         2.04370060e+00, -7.32120954e-02,  1.00464510e-01,
         5.90541024e-03, -4.64537797e-02,  2.63403480e-02,
         9.86721529e-02, -1.69185363e-02, -6.29591198e-03,
        -4.65162536e-03, -1.13423341e-01,  5.91680762e-02,
         1.93840000e-02,  1.12958376e-01,  2.30837297e-02,
         3.86322509e-02,  1.69741666e-02, -2.90049459e-02,
        -2.74171144e-01,  1.29342489e-01,  2.02305033e-02,
         3.75403275e-03,  1.16042801e-01, -2.28879294e-01,
         2.83663707e-01,  1.32694664e-01,  3.69682126e-01,
         2.45643008e-02,  2.14791215e-01,  8.08917815e-02,
         3.04522036e-01,  2.46486660e-01, -6.64100148e-02,
         4.77688063e-01],
       [-3.67948376e-01,  3.01965595e-01, -6.62385652e-01,
         2.35754008e-01,  1.31640672e-01,  3.44482471e-02,
         6.78018961e-01,  1.68961264e-02,  8.13950093e-02,
        -3.69199265e-02, -1.25

In [175]:
p_values_rank

array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0.7])

In [153]:
for i rank in enumerate(ranks):
    

SyntaxError: invalid syntax (1839014537.py, line 1)

In [13]:
dds = DeseqDataSet(
    table_20genes,
    metadata_CD,
    design_factors="disease",  # compare samples based on the "disease"
    # column ("B" vs "A")
    refit_cooks=True,
    n_cpus=8,
)
dds.deseq2()
stat_res = DeseqStats(dds, n_cpus=8)
    # Get the test statistics for each gene
res = stat_res.summary()

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.01 seconds.

Fitting dispersion trend curve...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Refitting 2 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.00 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.989640e+06,-0.414489,0.132655,-3.124574,1.780625e-03,0.003957
ko:K00002,9.003372e+05,-0.168351,0.176373,-0.954514,3.398238e-01,0.399793
ko:K00003,1.725142e+07,0.249343,0.101834,2.448514,1.434468e-02,0.028689
ko:K00004,8.929586e+05,-0.688170,0.179465,-3.834557,1.257909e-04,0.000629
ko:K00005,7.161609e+06,-0.292383,0.091469,-3.196507,1.391023e-03,0.003478
ko:K00006,2.816531e+01,0.547728,0.768327,0.712884,4.759173e-01,0.514934
ko:K00007,1.713274e+04,-1.823705,0.359277,-5.076039,3.853841e-07,0.000004
ko:K00008,1.194929e+07,0.050600,0.085166,0.594135,5.524219e-01,0.552422
ko:K00009,6.538020e+06,0.156448,0.152223,1.027759,3.040632e-01,0.380079


In [20]:
stat_res.summary()

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.989640e+06,-0.414489,0.132655,-3.124574,1.780625e-03,0.003957
ko:K00002,9.003372e+05,-0.168351,0.176373,-0.954514,3.398238e-01,0.399793
ko:K00003,1.725142e+07,0.249343,0.101834,2.448514,1.434468e-02,0.028689
ko:K00004,8.929586e+05,-0.688170,0.179465,-3.834557,1.257909e-04,0.000629
ko:K00005,7.161609e+06,-0.292383,0.091469,-3.196507,1.391023e-03,0.003478
ko:K00006,2.816531e+01,0.547728,0.768327,0.712884,4.759173e-01,0.514934
ko:K00007,1.713274e+04,-1.823705,0.359277,-5.076039,3.853841e-07,0.000004
ko:K00008,1.194929e+07,0.050600,0.085166,0.594135,5.524219e-01,0.552422
ko:K00009,6.538020e+06,0.156448,0.152223,1.027759,3.040632e-01,0.380079


In [26]:
res = stat_res.results_df
log2_fold_changes = resu["log2FoldChange"]
log2_fold_changes

#OTU ID
ko:K00001   -0.414489
ko:K00002   -0.168351
ko:K00003    0.249343
ko:K00004   -0.688170
ko:K00005   -0.292383
ko:K00006    0.547728
ko:K00007   -1.823705
ko:K00008    0.050600
ko:K00009    0.156448
ko:K00010   -0.088207
ko:K00011    2.956654
ko:K00012    0.118701
ko:K00013    0.190960
ko:K00014    0.090001
ko:K00015   -0.478737
ko:K00016   -0.154750
ko:K00018    0.182504
ko:K00019   -0.604472
ko:K00020   -0.695566
ko:K00021    1.108008
Name: log2FoldChange, dtype: float64

In [110]:
sorted_indices = np.argsort(log2_fold_changes)[::-1]
sorted_log2_fold_changes = log2_fold_changes[sorted_indices]
sorted_log2_fold_changes
rank = np.argsort(sorted_log2_fold_changes)
rank

#OTU ID
ko:K00011    19
ko:K00021    18
ko:K00006    17
ko:K00003    16
ko:K00013    15
ko:K00018    14
ko:K00009    13
ko:K00012    12
ko:K00014    11
ko:K00008    10
ko:K00010     9
ko:K00016     8
ko:K00002     7
ko:K00005     6
ko:K00001     5
ko:K00015     4
ko:K00019     3
ko:K00004     2
ko:K00020     1
ko:K00007     0
Name: log2FoldChange, dtype: int64

In [89]:
ranks = np.argsort(sorted_log2_fold_changes)
for i in enumerate(ranks):
    if i[1] > len(ranks)/2:
        print(i[])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [78]:
np.mean(1)

1.0

In [77]:
ranks[0]

19

In [37]:
expected_ranks = np.zeros(len(ranks))
expected_ranks

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [38]:
np.random.permutation(ranks)

array([16,  8,  5,  3,  2, 15, 17,  9, 10, 13, 18,  4, 12,  1,  0,  7,  6,
       11, 19, 14])

In [50]:
num_permutations = 10
p_values_rank = np.zeros(len(ranks))
for i, rank in enumerate(ranks):
   if rank < len(ranks)/2:
      p_values_rank[i] = 2*np.sum(expected_ranks <= rank)/num_permutations
   else:
      p_values_rank[i] = 2*np.sum(expected_ranks >= rank)/num_permutations

In [54]:
p_values_rank

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4.])

In [ ]:
    
    # Initialize the null distribution of the test statistic
    null_distribution = np.zeros((num_permutations, counts.shape[1]))

    # Perform the permutations
    for i in range(num_permutations):
        # Shuffle the group labels
#         permuted_labels = np.random.permutation(group_labels)
#         permuted_labels = pd.DataFrame(permuted_labels)
        permuted_labels = np.random.permutation(group_labels['disease'])
        permuted_labels = pd.DataFrame(permuted_labels, index=group_labels.index)
        permuted_labels = pd.DataFrame(permuted_labels)
        permuted_labels.columns = ['disease']
        # Convert the permuted group labels to PyDESeq2 input format
        # Run the DESeq2 analysis with the permuted group labels
        counts.reindex(permuted_labels.index)
        permuted_dds = DeseqDataSet(
        counts, 
        permuted_labels, 
        design_factors="disease")
        permuted_dds.deseq2()
        permuted_stat_res = DeseqStats(permuted_dds, n_cpus=8)
        # Get the test statistics for each gene with the permuted group labels
        permuted_res = permuted_stat_res.summary()

        # Add the permuted test statistic to the null distribution
        null_distribution[i, :] = permuted_stat_res.results_df['log2FoldChange']

    # Calculate the p-values for each gene
    p_values = np.zeros(counts.shape[1])
    for i in range(counts.shape[1]):
        if stat_res.results_df['log2FoldChange'].values[i] > 0:
            p_values[i] = np.mean(null_distribution[:, i] > stat_res.results_df['log2FoldChange'].values[i])
        else:
            p_values[i] = np.mean(null_distribution[:, i] < stat_res.results_df['log2FoldChange'].values[i])

    return p_values

In [39]:
pydeseq2_permutation_test(table_20genes, metadata_CD, num_permutations=10)

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.02 seconds.

Fitting dispersion trend curve...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Refitting 2 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.01 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.989640e+06,-0.414489,0.132655,-3.124574,1.780625e-03,0.003957
ko:K00002,9.003372e+05,-0.168351,0.176373,-0.954514,3.398238e-01,0.399793
ko:K00003,1.725142e+07,0.249343,0.101834,2.448514,1.434468e-02,0.028689
ko:K00004,8.929586e+05,-0.688170,0.179465,-3.834557,1.257909e-04,0.000629
ko:K00005,7.161609e+06,-0.292383,0.091469,-3.196507,1.391023e-03,0.003478
ko:K00006,2.816531e+01,0.547728,0.768327,0.712884,4.759173e-01,0.514934
ko:K00007,1.713274e+04,-1.823705,0.359277,-5.076039,3.853841e-07,0.000004
ko:K00008,1.194929e+07,0.050600,0.085166,0.594135,5.524219e-01,0.552422
ko:K00009,6.538020e+06,0.156448,0.152223,1.027759,3.040632e-01,0.380079


Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.01 seconds.

Fitting dispersion trend curve...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Refitting 4 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.60 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.501453e+06,0.059275,0.107912,0.549287,0.582808,0.832584
ko:K00002,9.003372e+05,0.127828,0.176750,0.723213,0.469549,0.783093
ko:K00003,1.725142e+07,-0.098735,0.104168,-0.947839,0.343211,0.762692
ko:K00004,8.929586e+05,0.370655,0.186972,1.982415,0.047433,0.247968
ko:K00005,7.161609e+06,0.064742,0.095457,0.678229,0.497627,0.783093
ko:K00006,2.816531e+01,1.072599,0.762459,1.406763,0.159498,0.455708
ko:K00007,1.971395e+04,0.316787,0.399632,0.792697,0.427954,0.783093
ko:K00008,1.194929e+07,-0.035104,0.085905,-0.408635,0.682807,0.879465
ko:K00009,6.538020e+06,-0.100838,0.152697,-0.660379,0.509011,0.783093


Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.01 seconds.

Fitting dispersion trend curve...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Refitting 4 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.19 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


...

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.01 seconds.

Fitting dispersion trend curve...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.37 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Refitting 4 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.13 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.501453e+06,-0.280618,0.104777,-2.678238,0.007401,0.049340
ko:K00002,9.003372e+05,0.055458,0.177067,0.313204,0.754126,0.859660
ko:K00003,1.725142e+07,0.040251,0.103814,0.387720,0.698223,0.859660
ko:K00004,8.929586e+05,-0.093768,0.189739,-0.494196,0.621167,0.859660
ko:K00005,7.161609e+06,0.172387,0.094282,1.828424,0.067486,0.337430
ko:K00006,5.209458e+01,-3.072788,0.770103,-3.990101,0.000066,0.001321
ko:K00007,1.971395e+04,0.422172,0.398911,1.058312,0.289913,0.859660
ko:K00008,1.194929e+07,0.015833,0.086004,0.184099,0.853936,0.859660
ko:K00009,6.538020e+06,-0.142760,0.152431,-0.936560,0.348985,0.859660


Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.01 seconds.

Fitting dispersion trend curve...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.02 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Refitting 4 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.48 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs CD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
#OTU ID,,,,,,
ko:K00001,6.501453e+06,-0.215928,0.106099,-2.035158,0.041835,0.209175
ko:K00002,9.003372e+05,0.132826,0.176669,0.751837,0.452149,0.753582
ko:K00003,1.725142e+07,0.017941,0.104579,0.171551,0.863790,0.909253
ko:K00004,8.929586e+05,-0.251826,0.188507,-1.335902,0.181581,0.412536
ko:K00005,7.161609e+06,-0.125630,0.094917,-1.323585,0.185641,0.412536
ko:K00006,4.208974e+01,-1.323511,0.794122,-1.666635,0.095587,0.273106
ko:K00007,1.407568e+04,-0.948269,0.359178,-2.640107,0.008288,0.082880
ko:K00008,1.194929e+07,0.054838,0.085824,0.638964,0.522847,0.804379
ko:K00009,6.538020e+06,-0.258772,0.151076,-1.712864,0.086738,0.273106


array([0. , 0. , 0. , 0. , 0. , 0.4, 0. , 0.3, 0.2, 0.3, 0. , 0.1, 0. ,
       0.1, 0. , 0.1, 0.1, 0. , 0. , 0. ])